In [1]:
import uproot4
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from matplotlib import cm
from matplotlib.colors import LogNorm, Normalize
import os
import numpy as np
import pandas as pd
import itertools

from event_display import  build_calorimeter_section

In [2]:
pics_prefix = os.path.join("plots", "v1_")

In [3]:
from geant3_parser import build_train_set
from geant3_parser import Geant3DataFile

file_name = os.path.join('data', 'shower_geant3_new.dat')
data_file = Geant3DataFile(file_name, skip_lines=3)
input_data, true_e, sum_e = build_train_set(data_file, 10000, add_real_xy=False, normalize=True)

# Remove x,y
input_data = input_data[:,2:]

# Reshape to events x 11 x 11 x 1 form
input_data = np.reshape(input_data, (len(input_data), 11, 11, 1))  # -1 => autodetermine

# Pad with 1 row and column of zeroes, so it divides by 2, events x 12 x 12 x 1 now
input_data = np.pad(input_data, ((0,0), (0,1), (0,1), (0,0)), mode='constant', constant_values=0)

# printout shape
print(f"Inputs shape new = {np.shape(input_data)}")

Time open 0.2693514823913574
Time split 0.24146318435668945
Inputs shape new = (10000, 12, 12, 1)


In [4]:
# finds loc of max value
'''row, col, max = 0, 0, 0
for i, c in enumerate(input_data[0]):
    for j, val in enumerate(c):
        if val > max:
            max = val
            row = j
            col = i

# sets box to be around max val
r_ma = row + 1
r_mi = row - 1
c_ma = col + 1
c_mi = col - 1

# gets outside box energy
outside_e = 0
for i, c in enumerate(input_data[0]):
    for j, val in enumerate(c):
        if i < c_mi or i > c_ma or j < r_mi or j > r_ma:
            outside_e += val
            input_data[0][i][j] = 0

added_e = float(outside_e / 9) # outside energy / num cells left 
# adds extra energy to all cells left
for i, c in enumerate(input_data[0]):
    for j, val in enumerate(c):
        if i >= c_mi and i <= c_ma and j >= r_mi and j <= r_ma:
            input_data[0][i][j] += added_e'''

'row, col, max = 0, 0, 0\nfor i, c in enumerate(input_data[0]):\n    for j, val in enumerate(c):\n        if val > max:\n            max = val\n            row = j\n            col = i\n\n# sets box to be around max val\nr_ma = row + 1\nr_mi = row - 1\nc_ma = col + 1\nc_mi = col - 1\n\n# gets outside box energy\noutside_e = 0\nfor i, c in enumerate(input_data[0]):\n    for j, val in enumerate(c):\n        if i < c_mi or i > c_ma or j < r_mi or j > r_ma:\n            outside_e += val\n            input_data[0][i][j] = 0\n\nadded_e = float(outside_e / 9) # outside energy / num cells left \n# adds extra energy to all cells left\nfor i, c in enumerate(input_data[0]):\n    for j, val in enumerate(c):\n        if i >= c_mi and i <= c_ma and j >= r_mi and j <= r_ma:\n            input_data[0][i][j] += added_e'

In [5]:
def display_event(event_data, fig=None, ax=None, cal_size=12, cell_size=1):
    # constants
    size_x = cell_size
    size_y = cell_size
    dx = size_x / 2.0
    dy = size_y / 2.0

    # go through all cells and calculate their centers
    centers = np.arange(-cal_size/2.0 + cell_size/2, cal_size/2 + cell_size/2, 1)
    positions = []
    for y_iter in range(cal_size):
        for x_iter in range(cal_size):
            positions.append((centers[x_iter], centers[y_iter]))

    # plot calorimeter with empty cells
    if not fig or not ax:
        fig, ax = plt.subplots()
    build_calorimeter_section(ax, positions, 1, 1)

    # Create a heat map
    norm = LogNorm()
    norm.autoscale(event_data)
    cmap = cm.get_cmap('inferno')

    # Convert data to rectangular patches    
    module_rects = []
    for y_iter in range(cal_size):
        for x_iter in range(cal_size):
            x = centers[x_iter]
            y = centers[y_iter]
            weight = event_data[x_iter][y_iter][0]
            #print(x,y,weight)
            patch = patches.Rectangle((x-dx, y-dy), size_x, size_y, edgecolor='black', facecolor=cmap(norm(weight)))
            module_rects.append(patch)

    # plot rectangles with data
    col = PatchCollection(module_rects, match_original=True)
    ax.add_collection(col)

    # plot heatmap legend
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(event_data)
    fig.colorbar(sm)
    
    return fig, ax


def display_event_comparison(left_data, right_data, fig=None, cal_size=12, cell_size=1):
    # constants
    size_x = cell_size
    size_y = cell_size
    dx = size_x / 2.0
    dy = size_y / 2.0

    # go through all cells and calculate their centers
    centers = np.arange(-cal_size/2.0 + cell_size/2, cal_size/2 + cell_size/2, 1)
    positions = []
    for y_iter in range(cal_size):
        for x_iter in range(cal_size):
            positions.append((centers[x_iter], centers[y_iter]))

    # plot calorimeter with empty cells
    if not fig:
        fig = plt.figure(figsize=(12,9))

    ax_left, ax_right = fig.subplots(1, 2)
    build_calorimeter_section(ax_left, positions, 1, 1)
    build_calorimeter_section(ax_right, positions, 1, 1)

    # Create a heat map
    norm = LogNorm()    
    norm.autoscale(np.vstack((left_data,right_data)))
    cmap = cm.get_cmap('inferno')

    def display_event_values(data, ax):
        # Convert data to rectangular patches    
        module_rects = []
        for y_iter in range(cal_size):
            for x_iter in range(cal_size):
                x = centers[x_iter]
                y = centers[y_iter]
                weight = data[x_iter][y_iter][0]
                #print(x,y,weight)
                patch = patches.Rectangle((x-dx, y-dy), size_x, size_y, edgecolor='black', facecolor=cmap(norm(weight)))
                module_rects.append(patch)

        # plot rectangles with data
        col = PatchCollection(module_rects, match_original=True)
        ax.add_collection(col)

    display_event_values(left_data, ax_left)
    display_event_values(right_data, ax_right)

    # plot heatmap legend
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(np.vstack((left_data,right_data)))
    fig.colorbar(sm, orientation="horizontal", ax=[ax_left,ax_right], extend="both")

    ax_left.set_aspect('equal', 'box')
    ax_right.set_aspect('equal', 'box')
    
    return fig, ax_left, ax_right

def get_bin_centers(bins):
    """Calculates bin centers out of bin boundaries"""
    assert len(bins) >= 2
    return bins[:-1] + (bins[1:] - bins[:-1]) / 2

# Prints 11x11 cells event
def print_event(table):
    if not len(table):
        print("EMPTY TABLE")
        return
    
    split_line = ""
    for irow, row in enumerate(table):
        if irow == 0:
            # First row => making title
            col_names = "ROW   " +  " ".join([f"{column_num:<5}" for column_num in range(len(row))])
            spaces = int((len(col_names) - len("COLUMNS"))/2)
            header = "{0}COLUMNS{0}".format(spaces*" ")
            split_line = "-"*len(col_names)
            print()            
            print(header)
            print(col_names)
            print(split_line)
        cells = f"{irow:<4}| " + " ".join([f"{cell[0]*11:<5.2}" for cell in row])
        print(cells)

    # Footer
    print(split_line)

In [6]:
# Load NN model
from keras.models import load_model
model = load_model('g3_autoencoder_conv_no_xy.hd5')

# Run ML on data
ml_events = model.predict(input_data)

2022-05-18 17:03:56.998801: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


OSError: SavedModel file does not exist at: g3_autoencoder_conv_no_xy.hd5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
true_event = np.exp(input_data[0]*11)
ml_event = np.exp(ml_events[0]*11)
ml_event[ml_event<2] = 0
true_event[true_event<1.5] = 0
display_event_comparison(true_event, ml_event)
print(f"max true {np.max(true_event)} max eval {np.max(ml_event)}")
print(f"sum true {np.sum(true_event)} sum eval {np.sum(ml_event)}")

In [ ]:
def data_to_energies(data):
    # We took log(hit) to make hits linear. /11 as a scale. Now reverting it    
    denorm_data = np.exp(data*11)

    # Get rid of array form of cells
    # For ML you have arrays like 
    # [num_pictures][x_pixels][y_pixels][colors]  where colors=3 for RGB,  colors=1 for BW
    # We use colors=1 (we just have an energy deposit in a cell) and we have num_pictures=num_events, [xy]_pixels=[xy]_cells
    # No we want to remove this useless [1]
    # (len(data), 12, 12, 1) => (len(data), 12, 12)
    reshaped = np.reshape(denorm_data, (len(data), 12, 12))

    # Sum along (x_pixels, y_pixels) axis => basicly summ everything in one event
    sums = np.sum(reshaped, axis=(1,2))
    return sums

sr_e = data_to_energies(input_data)             # SR = simple reco = energy from original data, summed together
ml_e = data_to_energies(ml_events)              # Energy from ML (sum everything)
true_e = 1000*np.reshape(true_e, len(input_data))    # True energy, 1000 to convert from GeV to

# create pandas table... for fun
pd = pd.DataFrame({"sr_e": sr_e, "ml_e": ml_e, "true_e":true_e})

In [ ]:
# using tuple unpacking for multiple Axes
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6))

ax1.hist(sr_e, bins=10)
ax2.hist(ml_e, bins=100)
ax3.hist2d(sr_e, ml_e/sr_e, bins=(15, 100))
plt.savefig(pics_prefix + "e_2dhist.png")
plt.show()


In [ ]:
# Build numpy 2d histogram
err_bins = np.arange(0.94, 1.03, 0.005)
e_bins = np.arange(0, 6000, 500)
err_bin_centers = get_bin_centers(err_bins)
e_bin_centers = get_bin_centers(e_bins)
de = ml_e/sr_e
dee_hist, de_bin_edges, e_bin_edges = np.histogram2d(sr_e, de, bins=[e_bins, err_bins])
e_bins_num, dee_bins_num = dee_hist.shape
e_bins_num, dee_bins_num, len(err_bin_centers)

In [ ]:
plot_columns = 4
plot_rows = int(e_bins_num / plot_columns) + 1
(int(e_bins_num/plot_columns) + 1)*plot_columns
print(f"Total subplots: {e_bins_num} plot rows={plot_rows} columns={plot_columns}")
fig, axes = plt.subplots(plot_rows, plot_columns, figsize=(plot_columns*4, plot_rows*4), sharey=True)

from statistics import NormalDist
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Fit fragment is taken from here
# https://gist.github.com/cpascual/a03d0d49ddd2c87d7e84b9f4ad2df466
def gauss(x, H, A, x0, sigma):
    return H + A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def gauss_fit(x, y):
    mean = sum(x * y) / sum(y)
    sigma = np.sqrt(sum(y * (x - mean) ** 2) / sum(y))
    popt, pcov = curve_fit(gauss, x, y, p0=[min(y), max(y), mean, sigma])
    return popt

# axes is a 2d array consisting of rows and colums [[...], [...], ...]
# flatten it as we need to feel it one by one
axes = list(itertools.chain.from_iterable(axes))
sigmas = []
dee_peaks = []
for i in range(e_bins_num):
    hist1d = dee_hist[i]
    ax = axes[i]
    # (!) using ax.step() for histograms
    # But commented ax.plot() to check ax.step() centers correctness
    # ax.plot(err_bin_centers, hist1d, "-o")    
    ax.step(err_bins[1:], hist1d)

    # Fit using Gaussian function
    H, A, x0, sigma = gauss_fit(err_bin_centers, hist1d)
    std = np.std(hist1d)    

    # Plot gauss
    smooth_gauss_x = np.arange(err_bins[0], err_bins[-1], (err_bins[-1]-err_bins[0])/1000)
    smooth_gauss_y = gauss(smooth_gauss_x, H, A, x0, sigma)
    ax.plot(smooth_gauss_x, smooth_gauss_y, '--r', label='fit')

    # titles
    ax.set_title(f"Eml/E for E={e_bins[i]}-{e_bins[i+1]}\n sgm={sigma:.3f} peak={x0:.4f}", wrap=True)  

    # Add our error
    sigmas.append(sigma)
    dee_peaks.append(x0)
    

plt.tight_layout()
plt.savefig(pics_prefix + "ml_div_true_fit")
plt.plot()

In [ ]:
plt.plot(e_bin_centers, sigmas, "--o")
plt.title("Sigma by energy")
plt.savefig(pics_prefix + "errors")

In [ ]:
sigma_div_e = sigmas/e_bin_centers

plt.plot(e_bin_centers, sigma_div_e, "--o")
plt.xlabel("Energy, MeV")
plt.savefig(pics_prefix + "errors_div_energy")

In [ ]:
def calo_error(x, a, b, c):
    return a/np.sqrt(x) + b/x + c

popt, pcov = curve_fit(calo_error, e_bin_centers, sigma_div_e, p0=[1, 1, 0])
a, b, c = popt
print(f"a={a} b={b} c={c}")

# Plot gauss
smooth_cal_x = np.arange(e_bin_centers[0], e_bin_centers[-1], (e_bin_centers[-1]-e_bin_centers[0])/1000)
smooth_cal_y = calo_error(smooth_cal_x, a, b, c)

fig, ax = plt.subplots()
ax.plot(e_bin_centers, sigma_div_e, "--o")
ax.plot(smooth_cal_x, smooth_cal_y, '--r', label='fit')


In [ ]:
plt.plot(e_bin_centers, dee_peaks , "--o")
plt.title("Mean MLE/E by energy")
plt.ylim(0.95,1.05)
plt.savefig(pics_prefix + "ml_div_true_peaks")